In [ ]:
import PyQt5
%matplotlib qt
import math
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
print("setup done")

In [ ]:
# df = pd.read_csv('./dataset/brazil_report.csv', sep=';', parse_dates=['Sale Date Time'])
# del df['Employee']
# df['Date'] = pd.to_datetime(df['Sale Date Time'])
# df = df.drop(columns=['Sale Date Time'])
# df = df.loc[(df['Client'] != 'Customer not informed')]
df = pd.read_pickle("./data_prod_code.pkl")
print("dataset Loaded")

In [ ]:
df.head()

In [ ]:
# from sklearn import preprocessing
# import pandas as pd

# le_cc = preprocessing.LabelEncoder()
# le_p = preprocessing.LabelEncoder()
# le_pc = preprocessing.LabelEncoder()
# le_pcost = preprocessing.LabelEncoder()
# df['Company Code'] = le_cc.fit_transform(df['Company Code'])
# df['Product'] = le_p.fit_transform(df['Product'])
# df['Product Category'] = le_pc.fit_transform(df['Product Category'])
# df['Product Cost'] = le_pcost.fit_transform(df['Product Cost'])

In [ ]:
df.head()

In [ ]:
dfprod = pd.DataFrame(df['ProductId'].unique())
dfprod.columns = ['ProductId']

# Recency
df_max_purchase = df.groupby('ProductId').Date.max().reset_index()
df_max_purchase.columns = ['ProductId','MaxPurchaseDate']
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max()+pd.DateOffset(1) - df_max_purchase['MaxPurchaseDate']).dt.days   
dfprod = pd.merge(dfprod, df_max_purchase[['ProductId','Recency']], on='ProductId')


# Frequency
dffreq = df.groupby('ProductId')['Order Number'].nunique().reset_index()
dffreq.columns = ['ProductId','Frequency']
dfprod = pd.merge(dfprod, dffreq, on='ProductId')


In [ ]:
# dfprod.sort_values(by='Frequency',ascending=False).head(20)

In [ ]:
dfprod['Rank_Frequency'] = pd.qcut( dfprod['Frequency'].rank(method='first'),q=2000, labels = range(1, 2001, 1))
dfprod['Rank_Frequency'] = pd.to_numeric(dfprod['Rank_Frequency'])

dfprod['Rank_Recency'] = pd.qcut( dfprod['Recency'].rank(method='first'),q=2000, labels = range(2001, 1, -1),)
dfprod['Rank_Recency'] = pd.to_numeric(dfprod['Rank_Recency'])


In [ ]:
dfprod.head()

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot()
ax.scatter(dfprod['Rank_Frequency'],dfprod['Rank_Recency'],cmap='brg',
           marker='x')
ax.set_xlabel('Frequency')
ax.set_ylabel('Recency')
plt.show()

In [ ]:
# inertia_err = []
# for k in range(0,20):
#     kmeans = KMeans(n_clusters=k+1, random_state = 42).fit(dfprod[['Rank_Frequency',
#                                                                      'Rank_Recency',
#                                                                     ]])
#     inertia_err.append(kmeans.inertia_)
    
# sns.pointplot(x=list(range(1,21)), y= inertia_err)
# plt.show()


In [ ]:
# we deice to go for 7 clusters
fin_clusters = 5

model = KMeans(n_clusters=fin_clusters, random_state = 42).fit(dfprod[['Rank_Frequency',
                                                                     'Rank_Recency',
                                                                    ]])
centers = kmeans.cluster_centers_



c=model.predict(dfprod[['Rank_Frequency','Rank_Recency',
                                        ]])
dfprod['Cluster'] = c

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(20,20))
# ax = fig.add_subplot()
# ax.scatter(dfprod['Rank_Frequency'],dfprod['Rank_Recency'],cmap='brg',
#            marker='x',c=dfprod['Cluster'],lable=dfprod['Cluster'])
# ax.set_xlabel('Frequency')
# ax.set_ylabel('Recency')

#plot data with seaborn
facet = sns.lmplot(data= dfprod, x='Rank_Frequency', y='Rank_Recency', hue='Cluster', 
                   fit_reg=False, legend=True, legend_out=True)
plt.legend()
plt.show()

if cluster 4 -> 9.5

if cluster 1 -> 7

if cluster 0 -> 5

if cluster 4 -> 8

if cluster 3 -> 2.5


In [ ]:
# rating is out of 1500
def rating_finder (row):
    rate = 0
    if(row['Cluster']==4):
        rate=1400
    if(row['Cluster']==1):
        rate=1050
    if(row['Cluster']==0):
        rate=750
    if(row['Cluster']==2):
        rate=1200
    if(row['Cluster']==3):
        rate=450
    return rate

In [ ]:
dfprod['Rating_rf'] =dfprod.apply (lambda row: rating_finder(row), axis=1)
# df['ProductId']=df.apply (lambda row: product_id_finder(row), axis=1)

In [ ]:
# rating is out of 1500
def rating_adder (row):
    rate = row['Rating_rf']+row['Rank_Frequency']+row['Rank_Recency']
    return rate

In [ ]:
dfprod['Rating_rf_combined'] =dfprod.apply (lambda row: rating_adder(row), axis=1)

In [ ]:
dfprod['Rating_rf_combined'].describe()

In [ ]:
dfprod['Rating_rf_combined_fin']=dfprod['Rating_rf_combined_fin']/10

In [ ]:
dfprod['Rating_rf_combined_fin'].describe()

In [ ]:
dfprod['Rating_rf_combined_fin'] = pd.qcut( dfprod['Rating_rf_combined'].rank(method='first'),q=100, labels = range(1, 101, 1))
dfprod['Rating_rf_combined_fin'] = pd.to_numeric(dfprod['Rating_rf_combined_fin'])


In [ ]:
# Frequency
df = pd.merge(df, dfprod[['ProductId','Rating_rf_combined_fin']], on='ProductId')


In [ ]:
df

Test Code

In [ ]:
# def product_id_finder (row):
#     return str(row['Company Code'])+str(row['Product'])+str(row['Product Category'])+str(row['Product Cost'])

In [ ]:
# df['ProductId']=df.apply (lambda row: product_id_finder(row), axis=1)

In [ ]:
# df=df.reset_index()
# df =df.drop(['index'],axis=1)

In [ ]:
# len(df['ProductId'].unique())

In [ ]:
# df.head()

In [ ]:
# df['Company Code'] = le_cc.inverse_transform(df['Company Code'])
# df['Product'] = le_p.inverse_transform(df['Product'])
# df['Product Category'] = le_pc.inverse_transform(df['Product Category'])
# df['Product Cost'] = le_pcost.inverse_transform(df['Product Cost'])

Test code below

In [ ]:
# dfclient = pd.DataFrame(df['Client'].unique())
# dfclient.columns = ['Client']

# # Recency
# df_max_purchase = df.groupby('Client').Date.max().reset_index()
# df_max_purchase.columns = ['Client','MaxPurchaseDate']
# df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max()+pd.DateOffset(1) - df_max_purchase['MaxPurchaseDate']).dt.days   
# dfclient = pd.merge(dfclient, df_max_purchase[['Client','Recency']], on='Client')


# # Frequency
# dffreq = df.groupby('Client')['Order Number'].nunique().reset_index()
# dffreq.columns = ['Client','Frequency']
# dfclient = pd.merge(dfclient, dffreq, on='Client')


# #Monetary
# #calculate revenue for each customer
# dfrev = df.groupby('Client').Total.sum().reset_index()
# dfrev.columns = ['Client','Monetary']
# dfclient = pd.merge(dfclient, dfrev[['Client','Monetary']], on='Client')

# # Create a backup
# dfclient_bk = dfclient.copy() 

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# scale = MinMaxScaler()
# dfclient[['Recency','Frequency','Monetary']] = scale.fit_transform(dfclient[['Recency','Frequency','Monetary']])

In [ ]:
# # Create ranks
# dfclient['Rank_Recency'] = pd.qcut( dfclient['Recency'].rank(method='first'),q=2000, labels = range(2001, 1, -1),)
# dfclient['Rank_Recency'] = pd.to_numeric(dfclient['Rank_Recency'])

# dfclient['Rank_Frequency'] = pd.qcut( dfclient['Frequency'].rank(method='first'),q=2000, labels = range(1, 2001, 1))
# dfclient['Rank_Frequency'] = pd.to_numeric(dfclient['Rank_Frequency'])

# dfclient['Rank_Monetary'] = pd.qcut( dfclient['Monetary'].rank(method='first'),q=2000, labels = range(1, 2001, 1))
# dfclient['Rank_Monetary'] = pd.to_numeric(dfclient['Rank_Monetary'])